In [1]:
import os
# HOME = os.getcwd()
HOME = '/workspace/football'
print(HOME)

/workspace/football


In [2]:
from typing import Generator

import matplotlib.pyplot as plt
import numpy as np

import cv2

# %matplotlib inline 


def generate_frames(video_file: str) -> Generator[np.ndarray, None, None]:
    video = cv2.VideoCapture(video_file)

    while video.isOpened():
        success, frame = video.read()

        if not success:
            break

        yield frame

    video.release()


def plot_image(image: np.ndarray, size: int = 12) -> None:
    plt.figure(figsize=(size, size))
    plt.imshow(image[...,::-1])
    plt.show()


In [3]:
SOURCE_VIDEO_PATH = f"{HOME}/20_sec_test.mp4"
WEIGHTS_PATH = f"{HOME}/best.pt"

In [4]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', WEIGHTS_PATH, device=0)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-5 Python-3.10.11 torch-2.0.1+cu117 CUDA:0 (NVIDIA A40, 45449MiB)

Fusing layers... 
Model summary: 416 layers, 139999708 parameters, 0 gradients, 207.9 GFLOPs
Adding AutoShape... 


In [5]:
frame_num = 4
frame_iterator = iter(generate_frames(video_file=SOURCE_VIDEO_PATH))
for i, frame in enumerate(frame_iterator):
    if i>frame_num:
        break
frame_16 = frame

frame_num = 600
frame_iterator = iter(generate_frames(video_file=SOURCE_VIDEO_PATH))
for i, frame in enumerate(frame_iterator):
    if i>frame_num:
        break
frame_600 = frame


In [6]:
results_16 = model(frame_16, size=1280)
results_600 = model(frame_600, size=1280)


In [12]:
query = []
for i, bbox in enumerate(results_16.pred[0]):
    left, top, right, bottom = bbox[:4].round().int()
    query.append((frame_16[top:bottom,left:right,::-1],i))

gallery = []
for i, bbox in enumerate(results_600.pred[0]):
    left, top, right, bottom = bbox[:4].round().int()
    gallery.append(frame_600[top:bottom,left:right,::-1])

In [8]:
from torchreid.utils import Reidentificator
import torch
from torch.nn import functional as F
# from torchreid import metrics

In [9]:
reid = Reidentificator(
    model_path='/workspace/football/sportsreid/log/model/working_models/model.pth.tar-10',
    device='cuda',
)


Model: vit_b_16
- params: 57,692,928
- flops: 11,279,979,008
Successfully loaded pretrained weights from "/workspace/football/sportsreid/log/model/working_models/model.pth.tar-10"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [13]:
reid.set_query(query)

In [14]:
reid(gallery)

array([[ 1, 14,  9],
       [ 4, 19,  5],
       [16, 13,  5],
       [11, 13, 16],
       [ 9, 10, 18],
       [ 9, 14, 18],
       [17,  3,  7],
       [10,  1, 14],
       [10,  9,  1],
       [19, 15,  4],
       [15,  2, 16],
       [20,  9,  6],
       [20,  6,  9],
       [13,  5,  8],
       [13,  5, 11],
       [ 8,  2,  0],
       [ 9, 20,  1],
       [20, 14,  9],
       [ 0, 16, 15],
       [13, 16, 11],
       [21, 23, 12],
       [23, 12, 24]])